# Lab Assignment 4: Support Vector Machines

## Overview
In this assignment, you will implement and analyze a Support Vector Machine (SVM).

We've attached a dataset, `MNIST.mat`, located in `/home/jovyan/work`, containing a sample of the famous MNIST benchmark. In this lab, you will develop predictive models for this data set and write a report on your findings. You will have less guidance in this lab, as you have more experience in the programming environment and we want you to have more ownership of your work. If you have questions, please reach out early and often. 


### Roadmap
In this lab, you will answer five questions where we:

1. Train a SVM to perform binary classification with non-linear kernels.

2. Implement a predictive model.

3. Compare the performance of two voting schemes.

4. Discuss your strategy for hyperparameter tuning.

5. Generate multiclass confusion matricies for your model.

### Restriction
You are not allowed to import an SVM from, for instance, `scikit-learn`.
You may, however, use a library (such as `scipy.optimize.minimize` or `cvxopt.solvers.qp`) for the optimization process. The code to install the `cvxopt` library is included in the first code cell, if needed. 

### Goals of This Lab
The goals of this lab are to:

1. Give you experience constructing and analyzing SVMs.

2. Continue to hone your communication skills through a brief report.

### Expectations
All lab submissions are individual and every item you submit should be a reflection of your own work. You should have ownership over the entirety of any lab you submit in this course. While your work is your own, we understand that it can be helpful in learning machine learning to collaborate with your peers, which can range from high-level discussion of a problem to debugging. Having others look at our code encourages us to write code with readability in mind. In practice, we will never work in a silo, and being able to discuss these topics with others well is a valuable skill. When you collaborate with another student, please cite them appropriately and be respectful of sharing too much. The Academic Honor Principle applies.

We respectfully ask that in the interest in furthering your own understanding of the material, that you refrain from using generative AI to code for you. Your work should be your own and you should feel comfortable justifying each design decision you make. 

Please cite any outside sources you reference.

### Evaluation
You will be evaluated on the quality of your code and report. Your report must provide summaries of each method's performance and some additional details of your implementation. Compare the relative strengths and weaknesses of the methods based on both the experimental results and your understanding of the algorithms.

### What to Submit
Please submit the following:

1. A link to your notebook so the TAs can evaluate your code.

2. A brief write-up that answers the 5 questions posed in this lab and justifies your model. Ensure that any figures you create are accessible and easy to understand.

### Library Installation:
In this lab we will use the cvxopt library, whose documentation can be found here: https://cvxopt.org/userguide/index.html.

We are using version 1.3.2 of cvxopt. You need only run this command once a lab session, then you may comment out the command for future runs of the notebook.

In [1]:
!pip install cvxopt

In [2]:
import numpy as np
from scipy.io import loadmat
import cvxopt
import cvxopt.solvers
from cvxopt import matrix

### Accessing the Data
You can load the data with `scipy.io.loadmat`, which will return a Python dictionary containing the test and train data and labels.

In [3]:
##Import the data
mnist = loadmat('MNIST.mat')
test_samples = mnist['test_samples']
test_samples_labels = mnist['test_samples_labels']
train_samples = mnist['train_samples']
train_samples_labels = mnist['train_samples_labels']

## Question 1:
Develop code for training an SVM for binary classification with nonlinear kernels. You'll need to accomodate non-overlapping class distributions. One way to implement this is to maximize (7.32) subject to (7.33) and (7.34). It may be helpful to redefine these as matrix operations. Let ${1}\in\mathbb{R}^{N\times 1}$ be the vector whose entries are all 1's. Let $\mathbf{a}\in\mathbb{R}^{N\times 1}$ have entries $a_i$. Let $\mathbf{T}\in\mathbb{R}^{N\times N}$ be a diagonal matrix with $\mathbf{T}_{ii} = t_i$ on the diagonal. Then we can reformulate the objective to be

\begin{equation*}
\begin{aligned}
& \text{maximize}
& & \tilde{L}(\mathbf{a}) = {1}^{\mathrm{T}}\mathbf{a} - \frac{1}{2} \mathbf{a}^{\mathrm{T}} \mathbf{T}\mathbf{K} \mathbf{T}\mathbf{a} \\
& \text{subject to}
& & {1}^{\mathrm{T}} \mathbf{a} \preceq C \\
& & & {1}^{\mathrm{T}} \mathbf{a} \succeq 0 \\
& & & \mathbf{a}^{\mathrm{T}} \mathbf{t} = 0
\end{aligned}
\end{equation*}

The "$\preceq$" symbol here means element-wise comparison. This formulation is very close to what `cvxopt` expects.

Hint (`cvxopt` expects the following form):

\begin{equation*}
\begin{aligned}
& \text{minimize}
& & \tilde{L}(\mathbf{a}) = \frac{1}{2} \mathbf{a}^{\mathrm{T}} \mathbf{T}\mathbf{K} \mathbf{T}\mathbf{a} - {1}^{\mathrm{T}}\mathbf{a} \\
& \text{subject to}
& & G \mathbf{a} \preceq h \\
& & & {\mathbf{t}}^{\mathrm{T}}\mathbf{a} = 0
\end{aligned}
\end{equation*}

where $G$ is an $N\times N$ identity matrix ontop of $-1$ times an $N\times N$ identity matrix and $h \in\mathbb{R}^{2N}$ where the first $N$ entries are $C$ and the second $N$ enties are $0$.

## Question 2:
Develop code to predict the $\{-1,+1\}$ class for new data. To use the predictive model (7.13) you need to determine $b$, which can be done with (7.37).

In [4]:
def nonlinear_kernel(X, Y):
    """RBF kernel"""
    gamma=0.1
    X_norm=np.sum(X**2, axis=1)
    Y_norm= np.sum(Y**2, axis=1)
    K =X_norm[:, None] + Y_norm[None, :]-2 * np.dot(X, Y.T)
    return np.exp(-gamma *K)

class SVM(object):
    ###Binary SVM classifier with nonlinear kernel support.##
    def __init__(self, kernel=nonlinear_kernel, C=1.0):
        self.kernel= kernel #  kernel function
        self.C =C  #regularization parameter
        self.a =None #lagrange multipliers
        self.b= None  #bias
        self.sv_X =None #support vectors
        self.sv_y=None  #Support vector labels
        self.sv_a= None  #support vector coefficients

    def fit(self, X, y):
        ###Train the SVM using quadratic programming.
        y= y.astype(np.float64).flatten()
        n_samples=X.shape[0]
        
        #compute kernel matrix
        K =self.kernel(X,X)
        K_t =np.outer(y , y)*K
        
        #convert to CVXOPT matrices
        P=cvxopt.matrix(K_t)
        q=cvxopt.matrix(-np.ones(n_samples))
        G=cvxopt.matrix(np.vstack([np.eye(n_samples),-np.eye(n_samples)])) #np.eye, Return a 2-D array with ones on the diagonal and zeros elsewhere.
        h=cvxopt.matrix(np.hstack([self.C*np.ones(n_samples), np.zeros(n_samples)]))
        A=cvxopt.matrix(y.reshape(1,-1).astype(float))
        b =cvxopt.matrix(0.0)
        
        #solve QP problem
        solution=cvxopt.solvers.qp(P,q, G, h,A, b)
        a =np.ravel(solution['x'])
        
        #store support vector
        sv_mask =a >1e-5 #using 1e-5 to avoid problems with numeric limitation
        self.sv_X=X[sv_mask]
        self.sv_y=y[sv_mask]
        self.sv_a=a[sv_mask]
        
        #compute bias term
        margin_mask =(a> 1e-5)&(a <self.C)
        if np.any(margin_mask):
            K_sv =self.kernel(X,X[margin_mask])
            self.b=np.mean(y[margin_mask] -np.dot(a*y, K_sv))
        else:
            K_sv =self.kernel(X,self.sv_X)
            self.b=np.mean(self.sv_y- np.dot(self.sv_a*self.sv_y, K_sv))

    def decision_function(self, X):
       ###Compute signed distance to hyperplane.
        K_test =self.kernel(X,self.sv_X)
        return np.dot(K_test,self.sv_a*self.sv_y)+self.b

    def predict(self, X):
        return np.sign(self.decision_function(X)).astype(int)

## Question 3:
Using your implementation, compare multiclass classification performance of two different voting schemes:

* one versus rest
* one versus one

In [5]:
class OvRClassifier:
   ##One-vs-Rest classifier with accelerated training with sampling.##
    def __init__(self, C=1.0, kernel=nonlinear_kernel, sample_ratio=0.2, random_state=None):
        self.classifiers= {}
        self.classes =None #class labels
        self.C =C
        self.kernel= kernel
        self.sample_ratio=sample_ratio #partial of data to use (0.0-1.0)
        self.random_state= random_state

    def fit(self, X, y):
        ###Train with random subsampling.###
        self.classes= np.unique(y)
        np.random.seed(self.random_state)

        
        for cls in self.classes:
            #subsample training dataa
            n_samples= int(len(X)*self.sample_ratio)
            indices =  np.random.choice(len(X), n_samples, replace=False)
            X_sub, y_sub =X[indices], y[indices]
            
            #Train binary classifier
            y_bin=np.where(y_sub == cls, 1, -1)
            svm=SVM(kernel=self.kernel, C=self.C)
            svm.fit(X_sub, y_bin)
            self.classifiers[cls] = svm

    def predict(self, X):
        ###Predict using max decision value.
        scores=np.zeros((X.shape[0], len(self.classes)))
        for idx, cls in enumerate(self.classes):
            scores[:, idx] =self.classifiers[cls].decision_function(X)
        return self.classes[np.argmax(scores, axis=1)]

class OvOClassifier:
    """One-vs-One classifier with accelerated training with sampling."""
    def __init__(self, C=1.0, kernel=nonlinear_kernel, sample_ratio=0.2, random_state=None):
        self.classifiers =[]
        self.classes =None
        self.C =C
        self.kernel =kernel
        self.sample_ratio =sample_ratio
        self.random_state =random_state

    def fit(self, X, y):
        """Train with random subsampling."""
        self.classes = np.unique(y)
        np.random.seed(self.random_state)
        
        #generate all classs pairs
        n_classes = len(self.classes)
        for i in range(n_classes):
            for j in range(i+1, n_classes):
                cls_i, cls_j =self.classes[i], self.classes[j]
                
                #get samples for current pair
                mask =np.logical_or(y == cls_i, y == cls_j)
                X_pair, y_pair = X[mask], y[mask]
                
                #subsample pair data
                n_samples =int(len(X_pair)*self.sample_ratio)
                if n_samples ==0: continue
                indices = np.random.choice(len(X_pair), n_samples, replace=False)
                X_sub, y_sub = X_pair[indices], y_pair[indices]
                
                ##Train binary classifier
                y_bin = np.where(y_sub == cls_i, 1, -1)
                svm = SVM(kernel=self.kernel, C=self.C)
                svm.fit(X_sub, y_bin)
                self.classifiers.append((svm, cls_i, cls_j))

    def predict(self, X):
        """Predict using majority voting."""
        votes =np.zeros((X.shape[0], len(self.classes)))
        for svm, cls_i, cls_j in self.classifiers:
            pred =svm.predict(X)
            for i, label in enumerate(np.where(pred == 1, cls_i, cls_j)):
                votes[i, np.where(self.classes ==label)[0][0]] += 1
        return self.classes[np.argmax(votes, axis=1)]

X_train = mnist['train_samples']
y_train = mnist['train_samples_labels'].ravel().astype(int)
X_test = mnist['test_samples']
y_test = mnist['test_samples_labels'].ravel().astype(int)

cvxopt.solvers.options['show_progress'] = False #turns off the screen output during calls to the solvers.

# OvR with acceleration
ovr = OvRClassifier(C=10.0,sample_ratio=0.3,random_state=42)
ovr.fit(X_train, y_train)
print(f"OvR Accuracy: {np.mean(ovr.predict(X_test) ==y_test):.4f}")

#OvO with acceleration
ovo = OvOClassifier(C=10.0, sample_ratio=0.3, random_state=42)
ovo.fit(X_train, y_train)
print(f"OvO Accuracy: {np.mean(ovo.predict(X_test) ==y_test):.4f}")

OvR Accuracy: 0.8760
OvO Accuracy: 0.8780


## Question 4:
The parameter $C>0$ controls the tradeoff between the size of the margin and the slack variable penalty. It is analogous to the inverse of a regularization coefficient. Include in your report a brief discussion of how you found an appropriate value.

In [6]:
# Hint: Try using np.logspace for hyperparameter tuning

#candidate C values: [0.001, 0.01, 0.1, 1, 10, 100, 1000]
C_values = np.logspace(1, 3, 5)

#Randomly sample 20% of the training data
sample_ratio= 0.2 #adjust based on time constraints
n_samples= int(len(X_train) * sample_ratio)
sample_indices =np.random.choice(len(X_train), n_samples, replace=False)
X_tune =X_train[sample_indices]
y_tune =y_train[sample_indices]

#data split
split =int(0.8 * n_samples)  # 80% training, 20% validation
X_train_sub, X_val =X_tune[:split], X_tune[split:]
y_train_sub, y_val =y_tune[:split], y_tune[split:]

best_C = None
best_val_accuracy = 0

for C in C_values:
    #OvR classifier with current C
    model = OvRClassifier(C=C, kernel=nonlinear_kernel)
    #Train on the subsampled data
    model.fit(X_train_sub, y_train_sub)
    
    #evaluate the validation set
    val_pred = model.predict(X_val)
    val_accuracy = np.mean(val_pred == y_val)
    
    #update best C
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_C = C
        
cvxopt.solvers.options['show_progress'] = False #turns off the screen output during calls to the solvers.

print(f"Best C: {best_C:.4f}, Validation Accuracy: {best_val_accuracy:.4f}")


Best C: 10.0000, Validation Accuracy: 0.8250


## Question 5:
In addition to calculating percent accuracy, generate multiclass [confusion matrices](https://en.wikipedia.org/wiki/confusion_matrix) as part of your analysis.

In [7]:
def compute_confusion_matrix(y_true, y_pred, classes):
    #y_true:ground truth labels
    #y_pred:predicted labels
    #classes:unique class labels
    #cm :confusion matrix.(n_classes, n_classes)
    n_classes =len(classes)
    cm =np.zeros((n_classes, n_classes), dtype=int)
    class_to_idx ={cls: idx for idx, cls in enumerate(classes)}
    
    for true, pred in zip(y_true, y_pred):
        true_idx =class_to_idx[true]
        pred_idx =class_to_idx[pred]
        cm[true_idx][pred_idx] +=1
    return cm

def print_confusion_matrix(cm, classes):
    ##Print confusion matrix.
    header= "True \\ Pred | " + " ".join(f"{cls:^4}" for cls in classes)
    print(header)
    print("-" *len(header))
    for i, cls in enumerate(classes):
        row = f"Class{cls:^4} | " + " ".join(f"{count:^4}" for count in cm[i])
        print(row)

#get class labels
classes = np.unique(y_test)

#generate predictions
ovr_pred= ovr.predict(X_test)
ovo_pred =ovo.predict(X_test)

#compute confusion matrices
ovr_cm = compute_confusion_matrix(y_test, ovr_pred, classes)
ovo_cm =compute_confusion_matrix(y_test, ovo_pred, classes)

#print results
print("OvR Confusion Matrix:")
print_confusion_matrix(ovr_cm, classes)
print("\nOvO Confusion Matrix:")
print_confusion_matrix(ovo_cm, classes)

OvR Confusion Matrix:
True \ Pred |  0    1    2    3    4    5    6    7    8    9  
---------------------------------------------------------------
Class 0   |  84   0    0    0    0    1    1    0    0    0  
Class 1   |  0   122   0    0    0    0    0    0    0    0  
Class 2   |  0    1    98   1    0    0    1    3    7    2  
Class 3   |  0    0    2    94   0    8    3    5    0    3  
Class 4   |  0    1    0    0    97   0    5    0    0    5  
Class 5   |  3    0    1    5    1    74   0    4    4    0  
Class 6   |  2    0    0    1    0    5    78   0    1    0  
Class 7   |  0    6    3    2    3    0    0    81   0    4  
Class 8   |  1    0    2    3    3    3    0    3    68   3  
Class 9   |  0    1    0    1    3    2    0    3    2    80 

OvO Confusion Matrix:
True \ Pred |  0    1    2    3    4    5    6    7    8    9  
---------------------------------------------------------------
Class 0   |  82   0    1    1    0    1    1    0    0    0  
Class 1   |  0   

### What to Submit
Please submit the following:

1. A link to your notebook so the TAs can evaluate your code.

2. A brief write-up that answers the 5 questions posed in this lab and justifies your model. Ensure that any figures you create are accessible and easy to understand.